# Quantum Interference: Amplitude Manipulation for Computation

## Introduction

**What is quantum interference?**  
Interference is the phenomenon where quantum amplitudes combine before measurement, allowing them to constructively enhance or destructively cancel. This is impossible with classical probabilities, which can only add.

**Why interference matters for quantum computing**  
Interference is the key mechanism that gives quantum algorithms their power. By carefully designing quantum circuits, we can:
- Amplify amplitudes of correct answers (constructive interference)
- Cancel amplitudes of wrong answers (destructive interference)
- Solve problems with fewer queries than classical algorithms

**Classical wave analogy**  
You've seen interference with water waves or sound waves—peaks align (constructive) or peaks meet troughs (destructive). Quantum interference is similar but operates on probability amplitudes, which can be complex numbers.

**What we'll demonstrate**  
In this notebook, we'll:
1. Simulate classical wave interference patterns
2. Show quantum amplitude interference with negative values
3. Implement the Deutsch algorithm (simplest quantum algorithm)
4. Demonstrate quantum advantage through interference
5. (Optional) Run on real quantum hardware

**Key insight**  
Classical probabilities: $P_{total} = P_1 + P_2$ (always positive)  
Quantum amplitudes: $P_{total} = |\alpha_1 + \alpha_2|^2 \neq |\alpha_1|^2 + |\alpha_2|^2$ (can cancel!)

## Setup: Imports and Configuration

Let's import our tools and configure beautiful visualizations.

In [ ]:
# Quantum computing framework
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector

# Numerical and visualization libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Custom utilities for beautiful plots
import sys
sys.path.append('..')
from utils.plotting import (
    configure_beautiful_plots,
    plot_histogram_comparison,
    plot_statevector,
    COLORS
)

# Configure beautiful plotting style
configure_beautiful_plots()

# Set random seed for reproducibility
np.random.seed(42)

print("✅ All imports successful")
print("✅ Ready to explore quantum interference")

## Classical Wave Interference

**What we'll do**  
Simulate two classical sine waves interfering with each other, showing both constructive and destructive interference.

**Why**  
This establishes the classical intuition for interference. Water waves, sound waves, and light waves all exhibit this behavior. However, classical waves always have positive amplitudes when we measure intensity.

**How**  
Create two sine waves and add them together:
- Same phase → constructive interference (peaks align)
- Opposite phase → destructive interference (peaks meet troughs)

**Expected result**  
Visual demonstration that waves can add or cancel based on their phase relationship.

In [ ]:
# Generate classical sine waves
x = np.linspace(0, 4*np.pi, 1000)
wave1 = np.sin(x)
wave2_constructive = np.sin(x)  # Same phase
wave2_destructive = np.sin(x + np.pi)  # Opposite phase (π shift)

# Interference patterns
constructive_sum = wave1 + wave2_constructive
destructive_sum = wave1 + wave2_destructive

# Beautiful 2x2 visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10), dpi=150)

# Constructive interference
ax1.plot(x, wave1, label='Wave 1', color=COLORS['primary'], linewidth=2, alpha=0.7)
ax1.plot(x, wave2_constructive, label='Wave 2 (same phase)', 
         color=COLORS['secondary'], linewidth=2, linestyle='--', alpha=0.7)
ax1.set_title('Individual Waves (Same Phase)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Position', fontsize=11)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)
ax1.axhline(y=0, color='black', linewidth=0.5)

ax2.plot(x, constructive_sum, color=COLORS['success'], linewidth=2.5)
ax2.fill_between(x, 0, constructive_sum, alpha=0.3, color=COLORS['success'])
ax2.set_title('Constructive Interference (Amplitudes Add)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Position', fontsize=11)
ax2.set_ylabel('Amplitude', fontsize=11)
ax2.grid(alpha=0.3)
ax2.axhline(y=0, color='black', linewidth=0.5)
ax2.text(0.5, 0.95, 'Amplitude ≈ 2× stronger', 
         transform=ax2.transAxes, fontsize=11, 
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
         verticalalignment='top')

# Destructive interference
ax3.plot(x, wave1, label='Wave 1', color=COLORS['primary'], linewidth=2, alpha=0.7)
ax3.plot(x, wave2_destructive, label='Wave 2 (opposite phase)', 
         color=COLORS['accent'], linewidth=2, linestyle='--', alpha=0.7)
ax3.set_title('Individual Waves (Opposite Phase)', fontsize=14, fontweight='bold')
ax3.set_xlabel('Position', fontsize=11)
ax3.set_ylabel('Amplitude', fontsize=11)
ax3.legend(fontsize=10)
ax3.grid(alpha=0.3)
ax3.axhline(y=0, color='black', linewidth=0.5)

ax4.plot(x, destructive_sum, color=COLORS['classical'], linewidth=2.5)
ax4.fill_between(x, 0, destructive_sum, alpha=0.3, color=COLORS['classical'])
ax4.set_title('Destructive Interference (Amplitudes Cancel)', fontsize=14, fontweight='bold')
ax4.set_xlabel('Position', fontsize=11)
ax4.set_ylabel('Amplitude', fontsize=11)
ax4.grid(alpha=0.3)
ax4.axhline(y=0, color='black', linewidth=0.5)
ax4.text(0.5, 0.95, 'Amplitude ≈ 0 (complete cancellation)', 
         transform=ax4.transAxes, fontsize=11,
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
         verticalalignment='top')

plt.suptitle('Classical Wave Interference Patterns', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\nKey observation: Waves can add (constructive) or cancel (destructive) based on phase.")
print("However, when we measure intensity (amplitude²), it's always positive.")

## Classical Interference: Key Insights

The visualizations show how waves can combine:
- **Constructive**: Same phase → amplitudes add → stronger signal
- **Destructive**: Opposite phase → amplitudes cancel → weaker/no signal

**Important limitation**: Even though wave amplitudes can be negative, when we measure **intensity** (energy, probability), we square the amplitude, which is always positive:

$$I = A^2 \geq 0$$

Classical probabilities must always be positive and additive. Quantum mechanics changes this...

## Quantum Interference: H-Z-H Circuit

**What we'll do**  
Create two quantum circuits that demonstrate interference:
1. **H-Z-H**: Hadamard → Z gate (phase flip) → Hadamard
2. **H-H**: Hadamard → Hadamard

**Why**  
These circuits show how quantum amplitudes interfere:
- H-Z-H: Destructive interference on |0⟩ → guaranteed |1⟩ outcome
- H-H: Constructive interference on |0⟩ → guaranteed |0⟩ outcome

**How**  
We'll build both circuits, run them, and show that interference can give us **certainty** from superposition.

**Expected result**  
- H-Z-H: 100% probability of measuring |1⟩
- H-H: 100% probability of measuring |0⟩

**The magic**: Interference allows the "wrong" amplitude to be cancelled completely!

In [ ]:
# Circuit 1: H-Z-H (expect |1⟩ - destructive on |0⟩)
qc_hzh = QuantumCircuit(1, 1)
qc_hzh.h(0)  # Create superposition
qc_hzh.z(0)  # Apply Z gate (phase flip)
qc_hzh.h(0)  # Apply H again
qc_hzh.measure(0, 0)

# Circuit 2: H-H (expect |0⟩ - constructive on |0⟩)
qc_hh = QuantumCircuit(1, 1)
qc_hh.h(0)   # Create superposition
qc_hh.h(0)   # Apply H again
qc_hh.measure(0, 0)

# Draw both circuits
print("Circuit 1: H-Z-H (Destructive interference on |0⟩)")
display(qc_hzh.draw('mpl', style='iqp'))

print("\nCircuit 2: H-H (Constructive interference on |0⟩)")
display(qc_hh.draw('mpl', style='iqp'))

# Run both circuits
simulator = AerSimulator()

job_hzh = simulator.run(qc_hzh, shots=1000)
counts_hzh = job_hzh.result().get_counts()

job_hh = simulator.run(qc_hh, shots=1000)
counts_hh = job_hh.result().get_counts()

print("\n" + "="*60)
print("RESULTS")
print("="*60)
print(f"\nH-Z-H circuit: {counts_hzh}")
print(f"H-H circuit: {counts_hh}")
print("\n" + "="*60)

In [ ]:
# Beautiful side-by-side comparison
fig = plot_histogram_comparison(
    counts_hzh,
    counts_hh,
    title1='H-Z-H: Destructive on |0⟩',
    title2='H-H: Constructive on |0⟩',
    overall_title='Quantum Interference: Certainty from Superposition'
)
plt.show()

print("\n🎯 Key Result: Through interference, we get CERTAIN outcomes!")
print("   H-Z-H → Always |1⟩ (destructive interference cancelled |0⟩ amplitude)")
print("   H-H → Always |0⟩ (constructive interference doubled |0⟩ amplitude)")

## Understanding the Interference

Let's understand **why** these circuits give certain outcomes by examining the amplitudes.

### Mathematical Analysis

**Hadamard gate action:**
$$H|0\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$$
$$H|1\rangle = \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$$

**Z gate action:**
$$Z|0\rangle = |0\rangle$$
$$Z|1\rangle = -|1\rangle$$

**Circuit H-Z-H:**
1. Start: $|0\rangle$
2. After first H: $\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$
3. After Z gate: $\frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)$ (phase flip on |1⟩)
4. After second H: $\frac{1}{\sqrt{2}}[H|0\rangle + H(-|1\rangle)]$
   - $= \frac{1}{\sqrt{2}}\left[\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) - \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)\right]$
   - $= \frac{1}{2}[(|0\rangle + |1\rangle) - (|0\rangle - |1\rangle)]$
   - $= \frac{1}{2}[2|1\rangle] = |1\rangle$ ✓

**Circuit H-H:**
1. Start: $|0\rangle$
2. After first H: $\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$
3. After second H: $\frac{1}{\sqrt{2}}[H|0\rangle + H|1\rangle]$
   - $= \frac{1}{\sqrt{2}}\left[\frac{1}{\sqrt{2}}(|0\rangle + |1\rangle) + \frac{1}{\sqrt{2}}(|0\rangle - |1\rangle)\right]$
   - $= \frac{1}{2}[(|0\rangle + |1\rangle) + (|0\rangle - |1\rangle)]$
   - $= \frac{1}{2}[2|0\rangle] = |0\rangle$ ✓

**Key insight:** Notice how the $|1\rangle$ terms cancel in H-H due to **negative amplitude** in $H|1\rangle$, while in H-Z-H the $|0\rangle$ terms cancel. This destructive interference is impossible with classical probabilities!

## Amplitude Visualization: Seeing Negative Values

**What we'll do**  
Extract the statevector (before measurement) and visualize the amplitudes, including their signs.

**Why**  
This shows the crucial difference from classical probabilities: quantum amplitudes can be **negative** (or even complex), allowing cancellation.

**How**  
Create circuits without measurement, extract statevectors, plot real and imaginary parts.

**Expected result**  
We'll see negative amplitude values that enable destructive interference.

In [ ]:
# Create circuits WITHOUT measurement to see amplitudes
qc_hzh_sv = QuantumCircuit(1)
qc_hzh_sv.h(0)
qc_hzh_sv.z(0)
qc_hzh_sv.h(0)
state_hzh = Statevector(qc_hzh_sv)

qc_hh_sv = QuantumCircuit(1)
qc_hh_sv.h(0)
qc_hh_sv.h(0)
state_hh = Statevector(qc_hh_sv)

# Also show intermediate state after first H
qc_h_sv = QuantumCircuit(1)
qc_h_sv.h(0)
state_h = Statevector(qc_h_sv)

# Print amplitudes
print("State Amplitudes (complex values):")
print("="*60)
print(f"\nAfter H:     {state_h.data}")
print(f"After H-Z-H: {state_hzh.data}")
print(f"After H-H:   {state_hh.data}")
print("\n" + "="*60)
print("\nProbabilities (|amplitude|²):")
print(f"After H:     |0⟩: {np.abs(state_h.data[0])**2:.3f}, |1⟩: {np.abs(state_h.data[1])**2:.3f}")
print(f"After H-Z-H: |0⟩: {np.abs(state_hzh.data[0])**2:.3f}, |1⟩: {np.abs(state_hzh.data[1])**2:.3f}")
print(f"After H-H:   |0⟩: {np.abs(state_hh.data[0])**2:.3f}, |1⟩: {np.abs(state_hh.data[1])**2:.3f}")

**Important note**: The H-Z-H and H-H circuits produce purely real amplitudes. To see **complex amplitudes** with non-zero imaginary parts, we need gates like the S gate (phase gate) or T gate. Let's add an example with the S gate!

In [ ]:
# Create a circuit with complex amplitudes: H-S
qc_hs_sv = QuantumCircuit(1)
qc_hs_sv.h(0)
qc_hs_sv.s(0)  # S gate adds phase: |1⟩ → i|1⟩
state_hs = Statevector(qc_hs_sv)

print("Complex Amplitude Example: H-S Circuit")
print("="*60)
print(f"After H-S: {state_hs.data}")
print(f"\nBreakdown:")
print(f"  |0⟩ amplitude: {state_hs.data[0]:.4f}")
print(f"  |1⟩ amplitude: {state_hs.data[1]:.4f}")
print(f"\nNote: The |1⟩ amplitude has imaginary part i/√2 ≈ {np.imag(state_hs.data[1]):.4f}i")
print("="*60)

In [ ]:
# Beautiful amplitude bar plots - now with 3 examples including complex amplitudes
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6), dpi=150)

labels = ['|0⟩', '|1⟩']
x_pos = np.arange(len(labels))
width = 0.35

# H-Z-H amplitudes (real only)
real_hzh = np.real(state_hzh.data)
imag_hzh = np.imag(state_hzh.data)

bars1 = ax1.bar(x_pos - width/2, real_hzh, width, label='Real part', 
                color=COLORS['primary'], alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x_pos + width/2, imag_hzh, width, label='Imaginary part',
                color=COLORS['secondary'], alpha=0.8, edgecolor='black', linewidth=1.5)

ax1.axhline(y=0, color='black', linewidth=0.8)
ax1.set_title('H-Z-H: Real Amplitudes', fontsize=14, fontweight='bold')
ax1.set_ylabel('Amplitude Value', fontsize=12)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(labels, fontsize=12)
ax1.legend(fontsize=10)
ax1.grid(axis='y', alpha=0.3)
ax1.set_ylim(-1.2, 1.2)

# H-H amplitudes (real only)
real_hh = np.real(state_hh.data)
imag_hh = np.imag(state_hh.data)

bars3 = ax2.bar(x_pos - width/2, real_hh, width, label='Real part',
                color=COLORS['accent'], alpha=0.8, edgecolor='black', linewidth=1.5)
bars4 = ax2.bar(x_pos + width/2, imag_hh, width, label='Imaginary part',
                color=COLORS['quantum'], alpha=0.8, edgecolor='black', linewidth=1.5)

ax2.axhline(y=0, color='black', linewidth=0.8)
ax2.set_title('H-H: Real Amplitudes', fontsize=14, fontweight='bold')
ax2.set_ylabel('Amplitude Value', fontsize=12)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(labels, fontsize=12)
ax2.legend(fontsize=10)
ax2.grid(axis='y', alpha=0.3)
ax2.set_ylim(-1.2, 1.2)

# H-S amplitudes (complex!)
real_hs = np.real(state_hs.data)
imag_hs = np.imag(state_hs.data)

bars5 = ax3.bar(x_pos - width/2, real_hs, width, label='Real part',
                color=COLORS['success'], alpha=0.8, edgecolor='black', linewidth=1.5)
bars6 = ax3.bar(x_pos + width/2, imag_hs, width, label='Imaginary part',
                color=COLORS['classical'], alpha=0.8, edgecolor='black', linewidth=1.5)

ax3.axhline(y=0, color='black', linewidth=0.8)
ax3.set_title('H-S: Complex Amplitudes!', fontsize=14, fontweight='bold')
ax3.set_ylabel('Amplitude Value', fontsize=12)
ax3.set_xticks(x_pos)
ax3.set_xticklabels(labels, fontsize=12)
ax3.legend(fontsize=10)
ax3.grid(axis='y', alpha=0.3)
ax3.set_ylim(-1.2, 1.2)

# Add annotation for complex amplitude
ax3.annotate('Imaginary part ≠ 0!', 
            xy=(1, imag_hs[1]), xytext=(1.3, 0.5),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=11, color='red', fontweight='bold')

plt.suptitle('Quantum Amplitudes: Can Be Negative AND Complex', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

print("\n⚡ Critical observation: Amplitudes can be ZERO, POSITIVE, NEGATIVE, or COMPLEX")
print("   • H-Z-H and H-H produce purely REAL amplitudes (positive or negative)")
print("   • H-S produces COMPLEX amplitudes with non-zero imaginary part")
print("   • This allows destructive interference to completely cancel states")
print("   • Classical probabilities can never be negative or complex!")


## The Deutsch Algorithm: Quantum Advantage via Interference

**Problem**: Given a black-box function $f: \{0,1\} \rightarrow \{0,1\}$, determine if it's **constant** (same output for both inputs) or **balanced** (different outputs).

**Classical solution**: Must query the function **twice** (once for $f(0)$, once for $f(1)$) to determine the answer.

**Quantum solution**: The Deutsch algorithm solves this with **one query** using superposition and interference!

**How it works**:
1. Start with $|0\rangle|1\rangle$
2. Apply H gates to both qubits → create superposition
3. Apply the oracle function $U_f$
4. Apply H gate to first qubit
5. Measure first qubit:
   - Result |0⟩ → function is **constant**
   - Result |1⟩ → function is **balanced**

**The magic**: Interference amplifies the global property (constant vs balanced) while cancelling local details.

In [ ]:
# Oracle for constant function f(x) = 0
def deutsch_oracle_constant_0():
    """Oracle for f(x) = 0 (constant)"""
    qc = QuantumCircuit(2)
    # Do nothing - output is always 0
    return qc

# Oracle for constant function f(x) = 1
def deutsch_oracle_constant_1():
    """Oracle for f(x) = 1 (constant)"""
    qc = QuantumCircuit(2)
    qc.x(1)  # Flip output qubit
    return qc

# Oracle for balanced function f(x) = x
def deutsch_oracle_balanced_identity():
    """Oracle for f(x) = x (balanced)"""
    qc = QuantumCircuit(2)
    qc.cx(0, 1)  # CNOT: output = input XOR output
    return qc

# Oracle for balanced function f(x) = NOT x
def deutsch_oracle_balanced_not():
    """Oracle for f(x) = NOT x (balanced)"""
    qc = QuantumCircuit(2)
    qc.x(0)    # Flip input
    qc.cx(0, 1)  # CNOT
    qc.x(0)    # Flip back
    return qc

def deutsch_algorithm(oracle_function):
    """Implement Deutsch algorithm with given oracle"""
    # Initialize qubits: q0 = |0⟩, q1 = |1⟩
    qc = QuantumCircuit(2, 1)
    
    # Prepare |1⟩ state for second qubit
    qc.x(1)
    
    # Apply Hadamard gates
    qc.h(0)
    qc.h(1)
    
    # Apply oracle
    qc.compose(oracle_function(), inplace=True)
    
    # Apply final Hadamard to first qubit
    qc.h(0)
    
    # Measure first qubit
    qc.measure(0, 0)
    
    return qc

# Test all four oracles
oracles = [
    ("Constant f(x)=0", deutsch_oracle_constant_0),
    ("Constant f(x)=1", deutsch_oracle_constant_1),
    ("Balanced f(x)=x", deutsch_oracle_balanced_identity),
    ("Balanced f(x)=NOT x", deutsch_oracle_balanced_not)
]

print("Deutsch Algorithm: Testing all oracles\n")
print("="*70)

simulator = AerSimulator()
results = []

for name, oracle_func in oracles:
    qc = deutsch_algorithm(oracle_func)
    job = simulator.run(qc, shots=1000)
    counts = job.result().get_counts()
    
    # Determine result
    if '0' in counts and counts.get('0', 0) > 900:
        determination = "CONSTANT"
    else:
        determination = "BALANCED"
    
    results.append((name, counts, determination))
    print(f"{name:25} → Measurement: {counts} → {determination}")

print("="*70)
print("\n🎯 All oracles correctly identified with just ONE query each!")

In [ ]:
# Visualize one Deutsch circuit
print("Example: Deutsch Algorithm Circuit (Balanced oracle f(x)=x)")
example_circuit = deutsch_algorithm(deutsch_oracle_balanced_identity)
display(example_circuit.draw('mpl', style='iqp'))

print("\nCircuit breakdown:")
print("1. X gate on q1: Prepare |1⟩ state")
print("2. H gates: Create superposition on both qubits")
print("3. Oracle (CNOT): Apply the function f(x)")
print("4. H gate on q0: Enable interference")
print("5. Measure q0: Read the answer (0=constant, 1=balanced)")

In [ ]:
# Beautiful comparison of all four results
from utils.plotting import plot_multiple_histograms

data_list = [counts for _, counts, _ in results]
titles = [f"{name}\n({det})" for name, _, det in results]

fig = plot_multiple_histograms(
    data_list,
    titles,
    overall_title='Deutsch Algorithm: One Query Determines Constant vs Balanced',
    ncols=2
)
plt.show()

print("\n" + "="*70)
print("QUANTUM ADVANTAGE DEMONSTRATED")
print("="*70)
print("\nClassical approach: 2 queries required (must evaluate f(0) and f(1))")
print("Quantum approach: 1 query sufficient (interference reveals global property)")
print("\nHow interference helps:")
print("  • Superposition evaluates f(0) and f(1) simultaneously")
print("  • Interference amplifies constant/balanced property")
print("  • Measurement collapses to definite answer")
print("="*70)

## Hardware Execution: Real Quantum Computer (Optional)

**What we'll do**  
Run the Deutsch algorithm on real quantum hardware (Compute Canada Monarch).

**Why**  
Interference is very sensitive to noise. Real hardware will show some errors due to:
- Gate imperfections
- Decoherence during multi-gate operations
- Readout errors

**Expected**  
Results should still mostly correctly identify constant vs balanced, but with some noise (e.g., 95% accuracy instead of 100%).

In [ ]:
# Monarch hardware execution (placeholder)
from utils.monarch_config import MonarchConfig, print_hardware_info

print_hardware_info()

print("\n" + "⚠"*35)
print("HARDWARE EXECUTION PLACEHOLDER")
print("⚠"*35)
print("\nTo run Deutsch algorithm on Compute Canada Monarch:")
print("1. Configure credentials in utils/monarch_config.py")
print("2. Initialize MonarchConfig and connect to backend")
print("3. Submit Deutsch circuits for each oracle")
print("4. Compare simulator vs hardware results")
print("\nExpected hardware behavior:")
print("  • Constant oracles: ~95%+ accuracy (robust to noise)")
print("  • Balanced oracles: ~90%+ accuracy (more sensitive)")
print("  • Interference patterns may be partially degraded")
print("  • Overall: Still demonstrates quantum advantage")
print("⚠"*35)

## Summary: Quantum Interference Fundamentals

**What we learned**:

1. **Classical wave interference** vs **quantum amplitude interference**
   - Classical: Waves interfere, but intensity (probability) always positive
   - Quantum: Amplitudes can be negative/complex, enabling cancellation

2. **Quantum amplitudes behave differently**
   - Can be negative or complex (not just positive like probabilities)
   - Interfere before squaring: $|\alpha_1 + \alpha_2|^2 \neq |\alpha_1|^2 + |\alpha_2|^2$
   - Enable destructive interference (complete cancellation)

3. **H-X-H and H-H circuits demonstrate interference**
   - H-X-H: Destructive on |0⟩ → certain |1⟩ outcome
   - H-H: Constructive on |0⟩ → certain |0⟩ outcome
   - Certainty from superposition through interference

4. **Deutsch algorithm shows quantum advantage**
   - Determines constant vs balanced with 1 query (classical needs 2)
   - Uses superposition to query both inputs simultaneously
   - Uses interference to extract global property

**Key insight**: Interference is what gives quantum algorithms their power. By carefully designing quantum circuits, we manipulate amplitudes to:
- Amplify correct answers (constructive interference)
- Cancel wrong answers (destructive interference)
- Extract information more efficiently than classical algorithms

**Next steps**:

In the next notebook, we'll explore **quantum entanglement** - where interference between multiple qubits creates correlations impossible in classical physics. This enables even more powerful quantum algorithms and quantum communication protocols.

---

*"The double-slit experiment has in it the heart of quantum mechanics. In reality, it contains the only mystery."*  
— Richard Feynman

And that mystery is **interference**. 🌊✨